In [2]:
import numpy as np
from StringIO import StringIO
import pandas as pd
from __future__ import division
import urllib2,base64
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
import scipy as sp
import datetime
import math
import operator
import re
import os
import os.path
import urllib, json
import csv
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import LeavePLabelOut
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import r2_score
from scipy import stats
from sklearn.neighbors import DistanceMetric
from urllib2 import urlopen, Request
from operator import itemgetter
from scipy.spatial import KDTree
from numpy import linalg as LA
from calendar import monthrange
from sklearn import metrics
import tarfile
import dask
import glob
#from deepmoji.sentence_tokenizer import SentenceTokenizer
#from deepmoji.model_def import deepmoji_emojis
#from deepmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
#from deepmoji.model_def import deepmoji_transfer
#from deepmoji.global_variables import PRETRAINED_PATH
#from deepmoji.finetuning import (
#    load_benchmark,
#    finetune)

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import dask.multiprocessing
from dask import compute, delayed

import time
%matplotlib inline

In [3]:
YEAR="2013"

In [4]:
listing = glob.glob(os.path.join('./', '2013_*.out'))

In [5]:
files=sorted(listing, key=lambda name: int(name.split('_')[1]))

In [6]:
df_from_each_file = (pd.read_csv(f,
                 sep=",", #or delim_whitespace=True, #separator is whitespace
                 header=None, #no header
                 names=['JOY','FEAR','ANGER','SADNESS','DISGUST','SHAME','GUILT'] #set columns names joy, fear, anger, sadness, disgust, shame, guilt
                 )  for f in files)
emoji_df   = pd.concat(df_from_each_file, ignore_index=True)


In [7]:
emoji_df.shape

(40187237, 7)

In [9]:
ill_df=pd.read_csv('data/US_2013_integrated.csv',encoding='utf-8',index_col=0) 
YEAR=2013
#ill_df['timestamp']=pd.to_datetime(ill_df['timestamp'], errors='coerce')
ill_df.shape

/home/s132732/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(40187237, 15)

In [10]:
ill_df=ill_df.reset_index(drop=True)
ill_df.head()

,count,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,PRCP,HUM,CC,WIND,POP,month
0,3,34.121012,26,-117.885040,2013-01-01,"[wish, i, was, at, the, snow, or, rose, parade...",True,15.71,1.91,0.00,66.990005,63.699951,4.442998,3870.3170,1
1,2,34.046998,10,-117.908113,2013-01-01,"[happy, fuckin, new, year],[@nataliejocelyne, ...",True,15.91,2.30,0.00,66.990005,63.699951,4.442998,2680.2053,1
2,1,42.023269,28,-88.245266,2013-01-01,"[must, say, we, had, a, great, time, saying, p...",True,0.12,-13.60,0.00,62.250000,5.399902,4.986554,1476.1556,1
3,1,34.121425,5,-117.884373,2013-01-01,"[i, love, having, thick, hair]",True,15.71,1.91,0.00,66.990005,63.699951,4.442998,3870.3170,1
4,1,39.595677,20,-89.578959,2013-01-01,"[lastnight, i, met, a, girl, who, got, increas...",True,1.40,-7.52,4.39,87.240005,56.899902,5.610321,1190.8944,1


In [11]:
twe_emo_df = pd.concat([ill_df, emoji_df],axis=1)
twe_emo_df.head()

,count,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,PRCP,...,WIND,POP,month,JOY,FEAR,ANGER,SADNESS,DISGUST,SHAME,GUILT
0,3,34.121012,26,-117.885040,2013-01-01,"[wish, i, was, at, the, snow, or, rose, parade...",True,15.71,1.91,0.00,...,4.442998,3870.3170,1,0.058981,0.124937,0.046433,0.056929,0.039310,0.267973,0.405437
1,2,34.046998,10,-117.908113,2013-01-01,"[happy, fuckin, new, year],[@nataliejocelyne, ...",True,15.91,2.30,0.00,...,4.442998,2680.2053,1,0.318294,0.081209,0.095020,0.059575,0.087509,0.259487,0.098906
2,1,42.023269,28,-88.245266,2013-01-01,"[must, say, we, had, a, great, time, saying, p...",True,0.12,-13.60,0.00,...,4.986554,1476.1556,1,0.409712,0.091944,0.120421,0.049077,0.071539,0.148865,0.108442
3,1,34.121425,5,-117.884373,2013-01-01,"[i, love, having, thick, hair]",True,15.71,1.91,0.00,...,4.442998,3870.3170,1,0.609639,0.038057,0.051051,0.030337,0.068157,0.150984,0.051775
4,1,39.595677,20,-89.578959,2013-01-01,"[lastnight, i, met, a, girl, who, got, increas...",True,1.40,-7.52,4.39,...,5.610321,1190.8944,1,0.232204,0.036115,0.047113,0.022468,0.282932,0.315178,0.063990


In [22]:

pixelwidth=0.0416666666667 #PRISM cell size (800m)
width_50=pixelwidth*12.5
width_25=pixelwidth*6.25
### BOunding box for Illinois
westlon=-124.7844079
southlat=24.7433195
eastlon=-66.9513812
northlat=49.3457868

resol_lon= int(round(abs(westlon-eastlon)/width_50))
resol_lat= int(round(abs(southlat-northlat)/width_50))

n_lat, n_lon=(resol_lat, resol_lon)
lat_bins = np.linspace(southlat, northlat, n_lat)
lon_bins = np.linspace(westlon, eastlon, n_lon)

lat_labels=np.arange(resol_lat-1)
lon_labels=np.arange(resol_lon-1)

### Bin all observations
twe_emo_df = twe_emo_df.assign(
    LAT_cut=pd.cut(twe_emo_df.LAT,lat_bins,labels=False),
    LON_cut=pd.cut(twe_emo_df.LON,lon_bins,labels=False)
)

twe_emo_df=twe_emo_df.rename(index=str, columns={"count": "count_daily"})
twe_emo_df['timestamp']=pd.to_datetime(twe_emo_df['timestamp'], errors='coerce')
### Categorize on Urban or NOt
twe_emo_df.dropna(axis=0, how='any',inplace=True)
ill_urban_df=twe_emo_df[twe_emo_df['URBAN']==True]
ill_rural_df=twe_emo_df[twe_emo_df['URBAN']==False]


print "Spatial Bin on LAT/LON with resolution (n_lat, n_lon):",(resol_lat, resol_lon)
#print lat_bins
#print lon_bins
twe_emo_df.head()

Spatial Bin on LAT/LON with resolution (n_lat, n_lon): (47, 111)


,count_daily,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,PRCP,...,month,JOY,FEAR,ANGER,SADNESS,DISGUST,SHAME,GUILT,LAT_cut,LON_cut
0,3,34.121012,26,-117.885040,2013-01-01,"[wish, i, was, at, the, snow, or, rose, parade...",True,15.71,1.91,0.00,...,1,0.058981,0.124937,0.046433,0.056929,0.039310,0.267973,0.405437,17,13
1,2,34.046998,10,-117.908113,2013-01-01,"[happy, fuckin, new, year],[@nataliejocelyne, ...",True,15.91,2.30,0.00,...,1,0.318294,0.081209,0.095020,0.059575,0.087509,0.259487,0.098906,17,13
2,1,42.023269,28,-88.245266,2013-01-01,"[must, say, we, had, a, great, time, saying, p...",True,0.12,-13.60,0.00,...,1,0.409712,0.091944,0.120421,0.049077,0.071539,0.148865,0.108442,32,69
3,1,34.121425,5,-117.884373,2013-01-01,"[i, love, having, thick, hair]",True,15.71,1.91,0.00,...,1,0.609639,0.038057,0.051051,0.030337,0.068157,0.150984,0.051775,17,13
4,1,39.595677,20,-89.578959,2013-01-01,"[lastnight, i, met, a, girl, who, got, increas...",True,1.40,-7.52,4.39,...,1,0.232204,0.036115,0.047113,0.022468,0.282932,0.315178,0.063990,27,66


In [23]:
ill_urban_monthlygrouped_df=ill_urban_df.groupby([ill_urban_df.month,ill_urban_df.LAT_cut,ill_urban_df.LON_cut])['count_daily'].sum().to_frame().reset_index()
ill_rural_monthlygrouped_df=ill_rural_df.groupby([ill_rural_df.month,ill_rural_df.LAT_cut,ill_rural_df.LON_cut])['count_daily'].sum().to_frame().reset_index()

ill_urban_monthlygrouped_df=ill_urban_monthlygrouped_df.rename(index=str, columns={"count_daily": "monthly_cell_count_urban"})
ill_rural_monthlygrouped_df=ill_rural_monthlygrouped_df.rename(index=str, columns={"count_daily": "monthly_cell_count_rural"})

ill_urban_monthlygrouped_df['monthly_cell_daily_count'] = ill_urban_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_urban'] / monthrange(YEAR,int(row.month))[1], axis=1)
ill_rural_monthlygrouped_df['monthly_cell_daily_count'] = ill_rural_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_rural'] / monthrange(YEAR,int(row.month))[1], axis=1)

ill_merge_monthlygrouped_df=pd.merge(ill_urban_monthlygrouped_df, ill_rural_monthlygrouped_df, how='outer', on=['month','LAT_cut','LON_cut'], sort=True,
         suffixes=('_urban', '_rural'))

ill_merge_monthlygrouped_df.fillna(0).head() ##MONTHLY DAILY CELL COUNT

,month,LAT_cut,LON_cut,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,1,2,84,4.0,0.129032,0.0,0.000000
1,1,3,84,32.0,1.032258,2.0,0.064516
2,1,3,85,1.0,0.032258,0.0,0.000000
3,1,4,84,1.0,0.032258,0.0,0.000000
4,1,5,80,25.0,0.806452,0.0,0.000000


In [24]:
ill_urban_dailygrouped_df=ill_urban_df.groupby([ill_urban_df.timestamp,ill_urban_df.LAT_cut,ill_urban_df.LON_cut]).agg({'count_daily':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
ill_rural_dailygrouped_df=ill_rural_df.groupby([ill_rural_df.timestamp,ill_rural_df.LAT_cut,ill_rural_df.LON_cut]).agg({'count_daily':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
#ill_rural_dailygrouped_df=ill_rural_df.groupby([ill_rural_df.timestamp,ill_rural_df.LAT_cut,ill_rural_df.LON_cut])['count_daily'].sum().to_frame().reset_index()

ill_merge_dailygrouped_df=pd.merge(ill_urban_dailygrouped_df, ill_rural_dailygrouped_df, how='outer', on=['timestamp','LAT_cut','LON_cut','month'], sort=True, suffixes=('_urban', '_rural'))

ill_merge_daily_df=pd.merge(ill_merge_dailygrouped_df,ill_merge_monthlygrouped_df, how="left", on=['LAT_cut','LON_cut','month'])

ill_merge_daily_df=ill_merge_daily_df
#f, ax = plt.subplots(1, 1, sharey=True)
#f.set_size_inches(18.5, 10.5, forward=True)

#test=ill_merge_daily_df[ill_merge_daily_df['timestamp']=='2016-02-21']
#plt.scatter(test['LON_cut'], test['LAT_cut'], s=700, c=test['PRCP_rural'], cmap="Reds", alpha=0.5,edgecolors=None,linewidths=0)
ill_merge_daily_df['timestamp']=ill_merge_daily_df['timestamp'].dt.strftime("%Y-%m-%d")
ill_merge_daily_df.head()

,timestamp,LAT_cut,LON_cut,TMIN_urban,CC_urban,month,count_daily_urban,SADNESS_urban,GUILT_urban,DISGUST_urban,...,TMAX_rural,SHAME_rural,HUM_rural,ANGER_rural,PRCP_rural,WIND_rural,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,2013-01-01,7,82,6.27,72.500000,1,1.0,0.053592,0.143544,0.101983,...,NaN,NaN,NaN,NaN,NaN,NaN,35.0,1.129032,NaN,NaN
1,2013-01-01,10,55,10.13,98.500000,1,1.0,0.077064,0.173794,0.215457,...,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.290323,NaN,NaN
2,2013-01-01,10,81,4.42,47.899902,1,4.0,0.117408,0.104279,0.122613,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.129032,NaN,NaN
3,2013-01-01,13,72,NaN,NaN,1,NaN,NaN,NaN,NaN,...,15.8,0.163565,91.990005,0.224248,0.0,3.431441,NaN,NaN,4.0,0.129032
4,2013-01-01,17,12,3.71,63.699951,1,1.0,0.060122,0.082424,0.550773,...,NaN,NaN,NaN,NaN,NaN,NaN,46.0,1.483871,NaN,NaN


In [25]:
ill_count_daily_df=pd.read_csv('data/US_2013_50KMbinned.csv',encoding='utf-8',index_col=0)

In [26]:
print "Emo shape ", ill_merge_daily_df.shape
print "Count shape ", ill_count_daily_df.shape

Emo shape  (375403, 36)
Count shape  (375403, 32)


In [27]:
ill_count_daily_df.head()

,Unnamed: 0.1,timestamp,LAT_cut,LON_cut,count_daily_urban,TMIN_urban,CC_urban,HUM_urban,TMAX_urban,PRCP_urban,...,monthly_cell_daily_count_rural,TMAX_HIS_mean,TMIN_HIS_mean,PRCP_HIS_mean,HUM_HIS_mean,WIND_HIS_mean,CC_HIS_mean,ncount_daily_urban,ncount_daily_rural,cell_id
0,0,2013-01-01,7,82,1.0,6.27,72.500000,76.250000,24.600000,0.00,...,NaN,22.749334,11.352666,6.319666,78.954666,3.452101,56.969963,-0.129032,NaN,"(7, 82)"
1,1,2013-01-01,10,55,1.0,10.13,98.500000,99.990005,18.459999,11.71,...,NaN,16.726332,3.882667,1.888667,79.368996,5.774523,47.706642,0.709677,NaN,"(10, 55)"
2,2,2013-01-01,10,81,4.0,4.42,47.899902,81.740005,18.799999,0.00,...,NaN,20.448002,7.850999,6.331000,79.553665,4.280912,56.859943,3.870968,NaN,"(10, 81)"
3,3,2013-01-01,13,72,NaN,NaN,NaN,NaN,NaN,NaN,...,0.129032,15.387332,3.849667,8.030334,83.910995,4.704171,48.159962,NaN,1.870968,"(13, 72)"
4,4,2013-01-01,17,12,1.0,3.71,63.699951,66.990005,15.210000,0.00,...,NaN,NaN,NaN,NaN,58.499668,4.855064,35.849983,-0.483871,NaN,"(17, 12)"


In [28]:
cols_to_use = ill_merge_daily_df.columns.difference(ill_count_daily_df.columns)
cols_to_use = cols_to_use.tolist()+ ['timestamp', 'LAT_cut','LON_cut']
print cols_to_use

['ANGER_rural', 'ANGER_urban', 'DISGUST_rural', 'DISGUST_urban', 'FEAR_rural', 'FEAR_urban', 'GUILT_rural', 'GUILT_urban', 'JOY_rural', 'JOY_urban', 'SADNESS_rural', 'SADNESS_urban', 'SHAME_rural', 'SHAME_urban', 'timestamp', 'LAT_cut', 'LON_cut']


In [29]:
ill_emo_count_daily_df = pd.merge(ill_count_daily_df.drop(['Unnamed: 0.1'], axis=1), ill_merge_daily_df[cols_to_use], how='inner', on=['timestamp', 'LAT_cut','LON_cut'],validate="one_to_one")

In [30]:
print "Merge Shape ",ill_emo_count_daily_df.shape

Merge Shape  (375403, 45)


In [31]:
ill_emo_count_daily_df.to_csv('data/US_2013_EMO_50KMbinned.csv',encoding='utf-8')